In [1]:
import pandas as pd
rway_img_df_ok = pd.read_csv("collection_images_download.csv")

In [2]:
rway_img_df = rway_img_df_ok.copy()

# EXTRACT BRAND FROM IMAGE NAME
# Creazione delle nuove colonne
rway_img_df['brand'] = rway_img_df['image_name'].str.lower()
rway_img_df['brand'] = rway_img_df['brand'].str.split('couture').str[0]
rway_img_df['brand'] = rway_img_df['brand'].str.split('ready-to-wear').str[0]
rway_img_df['brand'] = rway_img_df['brand'].str.split('menswear').str[0]
rway_img_df['brand'] = rway_img_df['brand'].str.replace("-", " ")
rway_img_df['brand'] = rway_img_df['brand'].str.strip()

In [ ]:
# create a small dataframe with the brand in scope
brand_in_scope = ['versace', 'hermes']
rway_img_df_small = pd.DataFrame()

for i,row in enumerate(rway_img_df['brand']):
    if row in brand_in_scope:
        rway_img_df_small = rway_img_df_small.append(rway_img_df.iloc[i])
        
rway_img_df_small['image_path_no_bkgnd'] = rway_img_df_small['image_path'].astype(str)

# Execute 
# for each row in rway_img_df_small['image_path_no_bkgnd'] change the output path
for index, row in rway_img_df_small.iterrows():
    # Split the image path into base name and extension
    base_name, extension = row['image_path_no_bkgnd'].split('.')

    # Add "-no-bkgnd" to the base name
    new_base_name = f"{base_name}-no-bkgnd"

    # Concatenate the new base name with the extension
    new_image_path = f"{new_base_name}.{extension}"

    # Replace the original image path with the new image path
    rway_img_df_small.at[index, 'image_path_no_bkgnd'] = new_image_path


In [4]:

#! it's takes 105m for 1694 imgs (% of full sample size)
#! Time estimate per img 2secs

"""from rembg import remove
import os

path_prefix = "./imgs/images/images/"

# Assuming rway_img_df_small is a pandas DataFrame containing the image paths
# with a column named 'image_path_no_bkgnd'

for index, row in rway_img_df_small.iterrows():
    input_path = f"{path_prefix}{row['image_path']}"
    output_path = f"{path_prefix}{row['image_path_no_bkgnd']}"

    with open(input_path, 'rb') as i:
        with open(output_path, 'wb') as o:
            input_data = i.read()
            output_data = remove(input_data)
            o.write(output_data)"""


'from rembg import remove\nimport os\n\npath_prefix = "./imgs/images/images/"\n\n# Assuming rway_img_df_small is a pandas DataFrame containing the image paths\n# with a column named \'image_path_no_bkgnd\'\n\nfor index, row in rway_img_df_small.iterrows():\n    input_path = f"{path_prefix}{row[\'image_path\']}"\n    output_path = f"{path_prefix}{row[\'image_path_no_bkgnd\']}"\n\n    with open(input_path, \'rb\') as i:\n        with open(output_path, \'wb\') as o:\n            input_data = i.read()\n            output_data = remove(input_data)\n            o.write(output_data)'

In [4]:
# from small dataframe create train (90%) and test (10%) set
from sklearn.model_selection import train_test_split
train, test = train_test_split(rway_img_df_small, test_size=0.1, random_state=42)


In [5]:
X_train, X_val, y_train, y_val = train_test_split(rway_img_df_small['image_path_no_bkgnd'], rway_img_df_small['brand'], test_size=0.2, random_state=1234)


In [6]:
display(X_train[:1])

959    hermes-menswear-fall-winter-2023-paris/Hermes-...
Name: image_path_no_bkgnd, dtype: object

In [8]:
"""import cv2
import numpy as np

output_image = cv2.imread(output_path)

# Resize the image to 260x260 pixels
resized_image = cv2.resize(output_image, (260, 260))

# Display the final resized image with a white background
cv2.imshow('Resized Image with White Background', resized_image)
cv2.waitKey(0)
cv2.destroyAllWindows()"""


"import cv2\nimport numpy as np\n\noutput_image = cv2.imread(output_path)\n\n# Resize the image to 260x260 pixels\nresized_image = cv2.resize(output_image, (260, 260))\n\n# Display the final resized image with a white background\ncv2.imshow('Resized Image with White Background', resized_image)\ncv2.waitKey(0)\ncv2.destroyAllWindows()"

# Image Classification with Attention

In [8]:
import os
import numpy as np
import pandas as pd
from tensorflow.python.keras.layers import Dense

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Multiply
from sklearn.model_selection import train_test_split


In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Multiply, Input
from tensorflow.keras.models import Model
import numpy as np
import tensorflow as tf

# Load the image data and labels

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(rway_img_df_small['image_path_no_bkgnd'], rway_img_df_small['brand'], test_size=0.2, random_state=1234)

# Define image and label dimensions (img_height and img_width) to resize the images during preprocessing.
img_height, img_width = 224, 224

#Map the brand labels in y_train and y_val to their corresponding integer labels using the brand_to_label dictionary.
brand_to_label = {brand: i for i, brand in enumerate(np.unique(rway_img_df_small['brand']))}

# Map brand labels to integer labels
y_train = np.array([brand_to_label[brand] for brand in y_train])
y_val = np.array([brand_to_label[brand] for brand in y_val])

# Calculate the number of classes (num_classes) based on the unique labels.
num_classes = len(brand_to_label)

# Define the preprocess_image function to load, resize, and normalize the pixel values of the images.
def preprocess_image(image_path):
    image_path = './imgs/images/images/' + image_path  # Add the path prefix
    image = load_img(image_path, target_size=(img_height, img_width))
    image = img_to_array(image)
    image = image / 255.0  # Normalize pixel values between 0 and 1
    return image

#Preprocess the images in X_train and X_val by applying the preprocess_image function to each image path.
X_train = np.array([preprocess_image(path) for path in X_train])
# Convert the integer labels in y_train and y_val to one-hot encoded vectors using tf.keras.utils.to_categorical.
X_val = np.array([preprocess_image(path) for path in X_val])

In [4]:
# Convert labels to one-hot encoded vectors
y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=num_classes)

# Load the VGG16 model without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(img_height, img_width, 3)))

# Add attention mechanism
attention = Dense(1, activation='tanh')(base_model.output)
attention = GlobalAveragePooling2D()(attention)
attention = Multiply()([base_model.output, attention])

# Add the top classification layer by applying a dense layer with a tanh activation function, followed by global average pooling and element-wise multiplication.
output = GlobalAveragePooling2D()(attention)
# Use global average pooling and a dense layer with softmax activation for multi-class classification.
output = Dense(num_classes, activation='softmax')(output)

# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=32)

# Save the trained model
model.save('./image_classification_model.h5')  # Replace with your desired path and filename


FileNotFoundError: [Errno 2] No such file or directory: 'your_dataframe.csv'